# **Task2**

In [5]:
import pandas as pd
import torch
import torch.nn as nn
from transformers import BertTokenizer, BertModel
from torch.utils.data import DataLoader, WeightedRandomSampler
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.utils.class_weight import compute_class_weight
from sklearn.metrics import precision_score, recall_score, f1_score, classification_report
import time

# Specify the device to use
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Load the data
train_df = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/Dataset/train_en.tsv', sep='\t')
train_df = train_df.drop(columns=['solved_conflict'])

# Convert textual labels to numeric labels
label_map = {"SUBJ": 0, "OBJ": 1}
train_df['label'] = train_df['label'].map(label_map)

# Extract sentences and labels
sentences_train = train_df['sentence'].tolist()
labels_train = train_df['label'].tolist()

# Split the data: 70% train, 15% validation, 15% test
train_sentences, temp_sentences, train_labels, temp_labels = train_test_split(
    sentences_train, labels_train, test_size=0.3, random_state=42
)

# Split the 30% into 50% validation and 50% test (15% validation, 15% test)
val_sentences, test_sentences, val_labels, test_labels = train_test_split(
    temp_sentences, temp_labels, test_size=0.5, random_state=42
)

# Compute class weights based on the distribution
class_weights = compute_class_weight(
    class_weight='balanced',
    classes=np.unique(train_labels),
    y=np.array(train_labels)
)

# Adjust the weight of the OBJ class based on an adjustment factor
obj_adjustment_factor = 1.2
class_weights[1] *= obj_adjustment_factor

# Convert the weights to a Tensor
class_weights = torch.tensor(class_weights, dtype=torch.float).to(device)

print(f"Class Weights: SUBJ={class_weights[0]:.4f}, OBJ={class_weights[1]:.4f}")

# Set up the tokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

class TextDataset(torch.utils.data.Dataset):
    def __init__(self, sentences, labels, tokenizer, max_len):
        self.sentences = sentences
        self.labels = labels
        self.tokenizer = tokenizer
        self.max_len = max_len

    def __len__(self):
        return len(self.sentences)

    def __getitem__(self, item):
        sentence = self.sentences[item]
        label = self.labels[item]
        encoding = self.tokenizer.encode_plus(
            sentence,
            add_special_tokens=True,
            max_length=self.max_len,
            padding='max_length',
            truncation=True,
            return_tensors='pt'
        )
        return {
            'input_ids': encoding['input_ids'].squeeze(0),
            'attention_mask': encoding['attention_mask'].squeeze(0),
            'labels': torch.tensor(label, dtype=torch.long),
            'sentence': sentence
        }

# Set up Dataset and DataLoader
max_len = 128
train_dataset = TextDataset(train_sentences, train_labels, tokenizer, max_len=max_len)
val_dataset = TextDataset(val_sentences, val_labels, tokenizer, max_len=max_len)
test_dataset = TextDataset(test_sentences, test_labels, tokenizer, max_len=max_len)

sample_weights = [class_weights[label] for label in train_labels]
train_sampler = WeightedRandomSampler(sample_weights, len(sample_weights), replacement=True)

train_loader = DataLoader(train_dataset, batch_size=32, sampler=train_sampler, num_workers=4)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False, num_workers=4)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False, num_workers=4)

class BERT_LSTM_Classifier(nn.Module):
    def __init__(self, hidden_dim, n_classes, num_layers=2, dropout=0.5):
        super(BERT_LSTM_Classifier, self).__init__()
        self.bert = BertModel.from_pretrained('bert-base-uncased')
        self.lstm = nn.LSTM(
            self.bert.config.hidden_size, hidden_dim, num_layers,
            batch_first=True, bidirectional=True, dropout=dropout
        )
        self.fc = nn.Linear(hidden_dim * 2, n_classes)
        self.dropout = nn.Dropout(dropout)

    def forward(self, input_ids, attention_mask):
        outputs = self.bert(input_ids, attention_mask=attention_mask)
        x, _ = self.lstm(outputs.last_hidden_state)
        x = x[:, -1, :]
        x = self.dropout(x)
        return self.fc(x)

learning_rates = [0.000005, 0.00001, 0.000008]
batch_sizes = [32, 64]
num_layers_list = [1, 2]
results = []

for lr in learning_rates:
    for batch_size in batch_sizes:
        for num_layers in num_layers_list:
            print(f"Training with LR={lr}, Batch Size={batch_size}, Num Layers={num_layers}")

            model = BERT_LSTM_Classifier(hidden_dim=256, n_classes=2, num_layers=num_layers)
            model.to(device)

            criterion = nn.CrossEntropyLoss(weight=class_weights)
            optimizer = torch.optim.AdamW(model.parameters(), lr=lr)
            scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=0.5, patience=2)

            best_val_accuracy, epochs_no_improve = 0.0, 0
            num_epochs = 15

            # Calculate total training time
            start_time = time.time()

            for epoch in range(num_epochs):
                model.train()
                total_loss, correct_preds, total_preds = 0, 0, 0

                for batch in train_loader:
                    input_ids = batch['input_ids'].to(device)
                    attention_mask = batch['attention_mask'].to(device)
                    labels = batch['labels'].to(device)

                    optimizer.zero_grad()
                    outputs = model(input_ids, attention_mask)
                    loss = criterion(outputs, labels)
                    loss.backward()
                    optimizer.step()

                    total_loss += loss.item()
                    _, preds = torch.max(outputs, dim=1)
                    correct_preds += torch.sum(preds == labels)
                    total_preds += labels.size(0)

                train_accuracy = correct_preds.double() / total_preds
                avg_loss = total_loss / len(train_loader)

                model.eval()
                val_correct, val_total = 0, 0
                true_labels, pred_labels = [], []
                with torch.no_grad():
                    for batch in val_loader:
                        input_ids = batch['input_ids'].to(device)
                        attention_mask = batch['attention_mask'].to(device)
                        labels = batch['labels'].to(device)
                        outputs = model(input_ids, attention_mask)
                        _, preds = torch.max(outputs, dim=1)
                        val_correct += torch.sum(preds == labels)
                        val_total += labels.size(0)
                        true_labels.extend(labels.cpu().numpy())
                        pred_labels.extend(preds.cpu().numpy())

                val_accuracy = val_correct.double() / val_total
                precision = precision_score(true_labels, pred_labels, average='weighted')
                recall = recall_score(true_labels, pred_labels, average='weighted')
                f1 = f1_score(true_labels, pred_labels, average='weighted')

                scheduler.step(avg_loss)

                print(f"Epoch {epoch + 1}/{num_epochs} - Loss: {avg_loss:.4f}, Train Acc: {train_accuracy:.4f}, "
                      f"Val Acc: {val_accuracy:.4f}, Precision: {precision:.4f}, Recall: {recall:.4f}, F1: {f1:.4f}")

                if val_accuracy > best_val_accuracy:
                    best_val_accuracy = val_accuracy
                    epochs_no_improve = 0
                else:
                    epochs_no_improve += 1
                    if epochs_no_improve >= 3:
                        print("Early stopping triggered!")
                        break

            # Measure elapsed time
            total_time = time.time() - start_time

            # Measure memory usage
            max_memory_allocated = torch.cuda.max_memory_allocated(device) / (1024 ** 2) if torch.cuda.is_available() else 0

            print(f"Training Time: {total_time:.2f} seconds")
            print(f"Max Memory Usage: {max_memory_allocated:.2f} MB")

            print(classification_report(true_labels, pred_labels, target_names=label_map.keys()))
            results.append({
                'Learning Rate': lr,
                'Batch Size': batch_size,
                'Num Layers': num_layers,
                'Best Train Accuracy': train_accuracy.item(),
                'Best Val Accuracy': best_val_accuracy.item(),
                'Precision': precision,
                'Recall': recall,
                'F1': f1,
                'Training Time (s)': total_time,
                'Max Memory Usage (MB)': max_memory_allocated
            })

# Convert results to DataFrame for easy inspection
results_df = pd.DataFrame(results)

# Save results to Excel file
results_df.to_excel('model_results.xlsx', index=False)

print("Results saved to 'model_results.xlsx'")

# Now evaluate the model on the test data
test_df = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/Dataset/test_en_gold.tsv', sep='\t')
test_df['label'] = test_df['label'].map(label_map)

# Extract sentences and labels for test
sentences_test = test_df['sentence'].tolist()
labels_test = test_df['label'].tolist()

# Set up DataLoader for test data
test_dataset = TextDataset(sentences_test, labels_test, tokenizer, max_len=max_len)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False, num_workers=4)

# Evaluate the model on test data
model.eval()
test_correct, test_total = 0, 0
test_true_labels, test_pred_labels = [], []

with torch.no_grad():
    for batch in test_loader:
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        labels = batch['labels'].to(device)
        outputs = model(input_ids, attention_mask)
        _, preds = torch.max(outputs, dim=1)
        test_correct += torch.sum(preds == labels)
        test_total += labels.size(0)
        test_true_labels.extend(labels.cpu().numpy())
        test_pred_labels.extend(preds.cpu().numpy())

# Calculate accuracy and other metrics for test data
test_accuracy = test_correct.double() / test_total
test_precision = precision_score(test_true_labels, test_pred_labels, average='weighted')
test_recall = recall_score(test_true_labels, test_pred_labels, average='weighted')
test_f1 = f1_score(test_true_labels, test_pred_labels, average='weighted')

print(f"Test Accuracy: {test_accuracy:.4f}, Precision: {test_precision:.4f}, Recall: {test_recall:.4f}, F1: {test_f1:.4f}")
print(classification_report(test_true_labels, test_pred_labels, target_names=label_map.keys()))


Class Weights: SUBJ=1.4381, OBJ=0.9198


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

...

Epoch 13/15 - Loss: 0.1060, Train Acc: 0.9897, Val Acc: 0.8387, Precision: 0.8386, Recall: 0.8387, F1: 0.8366


/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:617: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(


Epoch 14/15 - Loss: 0.1090, Train Acc: 0.9811, Val Acc: 0.8145, Precision: 0.8139, Recall: 0.8145, F1: 0.8142
Early stopping triggered!
Training Time: 205.70 seconds
Max Memory Usage: 4341.67 MB
              precision    recall  f1-score   support

        SUBJ       0.77      0.76      0.76        49
         OBJ       0.84      0.85      0.85        75

    accuracy                           0.81       124
   macro avg       0.81      0.80      0.81       124
weighted avg       0.81      0.81      0.81       124

Results saved to 'model_results.xlsx'


/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:617: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(


Test Accuracy: 0.7490, Precision: 0.7520, Recall: 0.7490, F1: 0.7490
              precision    recall  f1-score   support

        SUBJ       0.78      0.72      0.75       127
         OBJ       0.72      0.78      0.75       116

    accuracy                           0.75       243
   macro avg       0.75      0.75      0.75       243
weighted avg       0.75      0.75      0.75       243

